In [1]:
import sys
import pandas as pd
from absl import app
import tensorflow as tf

In [2]:
#train_filename = "../Criteo_x1/train.csv"
#val_filename = "/Desktop/Cornell/AIstudio/Datasets/Criteo_x1/valid.csv"
test_filename = "../Criteo_x1/test.csv"

df = pd.read_csv(test_filename)
df.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1,0.0,0.008292,0.11,0.10,0.160344,0.068,0.02,0.08,0.010,...,1528983,1528994,1534050,1536021,1536022,1934144,1934163,1934311,2022806,2024736
1,1,0.0,0.134328,0.02,0.30,0.067359,0.170,0.04,0.36,0.460,...,1528982,1529471,1533924,1536018,1707732,1934144,1934163,1935911,2022801,2022897
2,0,0.0,0.003317,0.00,0.00,0.077438,0.000,0.00,0.74,0.194,...,1528986,1529049,1533924,1536018,1536023,1934144,1934169,1934181,2022801,2022897
3,1,1.0,0.533997,0.00,0.08,0.000078,0.008,0.89,0.80,0.176,...,1528988,1529017,1533924,1536018,1536209,1934144,1934163,1934312,2022801,2022897
4,0,0.0,0.097844,0.02,0.00,0.000000,0.000,0.00,0.00,0.002,...,1528990,1529030,1533924,1536018,1536025,1934144,1934167,1934181,2022801,2022897


In [3]:
learning_rate = 0.0002282433105027466
hidden_layer_dims = [768, 256, 128]
BATCH_SIZE = 512
num_train_steps = 150000
num_eval_steps = 8634


LABEL_FEATURE = "label"
LABEL_FEATURE_TYPE = tf.float32
NUMERIC_FEATURES = ["I1", "I2", "I3", "I4", "I5", "I6", "I7", "I8", "I9", "I10", "I11", "I12", "I13"]
NUMERIC_FEATURE_TYPES = [tf.float32] * len(NUMERIC_FEATURES)
CATEGORICAL_FEATURES = ["C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", "C11", "C12", "C13", "C14", "C15",

"C16", "C17", "C18", "C19", "C20", "C21", "C22", "C23", "C24", "C25", "C26"]

CATEGORICAL_FEATURE_TYPES = [tf.int32] * len(CATEGORICAL_FEATURES)
CATEGORICAL_FEATURE_EMBEDDING_DIMENSION = 20
NUM_BINS = 10000

In [4]:

def get_dataset(file_pattern, is_test):
    dataset = tf.data.experimental.make_csv_dataset(
        file_pattern=file_pattern, 
        batch_size=BATCH_SIZE, 
        column_names=[LABEL_FEATURE] + NUMERIC_FEATURES + CATEGORICAL_FEATURES,
        column_defaults=[LABEL_FEATURE_TYPE] + NUMERIC_FEATURE_TYPES + CATEGORICAL_FEATURE_TYPES,
        label_name=LABEL_FEATURE,
        header=True,
        num_epochs=1 if is_test else None,
        shuffle=True,
        num_parallel_reads=16)
    return dataset

In [5]:
class M(tf.keras.Model):

   def __init__(self, layer_dims, layer_activations):
       super(M, self).__init__()

       self.Es = {}
       self.Hs = {}

       for name in CATEGORICAL_FEATURES:
           self.Hs[name] = tf.keras.layers.Hashing(NUM_BINS)
           self.Es[name] = tf.keras.layers.Embedding(
               NUM_BINS, CATEGORICAL_FEATURE_EMBEDDING_DIMENSION)

       self.Ls = []
       for dim, activation in zip(layer_dims, layer_activations):
           self.Ls += [tf.keras.layers.Dense(dim, activation=activation)]

   @tf.function
   def call(self, inputs):
       outputs = []
       for name in NUMERIC_FEATURES:
           output = tf.reshape(inputs[name], [-1, 1])
           outputs.append(output)
       for name in CATEGORICAL_FEATURES:
           output = self.Hs[name](inputs[name])
           output = self.Es[name](output)
           outputs.append(output)
       outputs = tf.keras.layers.concatenate(outputs)
       for L in self.Ls:
           outputs = L(outputs)
       return tf.math.sigmoid(outputs)


In [16]:

from absl import app, flags

FLAGS = flags.FLAGS


def main(argv):
    del argv

    train_dataset = get_dataset(train_filename, False)
    # print("DATASET", next(train_dataset.take(1).as_numpy_iterator()))
    # return

    val_dataset = get_dataset(val_filename, False)
    test_dataset = get_dataset(test_filename, True)

    model = M(
        layer_dims=hidden_layer_dims + [1],
        layer_activations=["relu"] * len(hidden_layer_dims) + [None],
    )
    optimizer = tf.keras.optimizers.legacy.Adam(
        learning_rate=learning_rate, clipnorm=100
    )
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )

    history = model.fit(
        x=train_dataset,
        epochs=1500,
        steps_per_epoch=100,
        verbose=2,
        # validation_data=eval_dataset,
        # validation_steps=100,
    )

    results = model.evaluate(test_dataset)
    print('results', results)


if __name__ == "__main__":
    filtered_argv = [arg for arg in sys.argv if not arg.startswith('-f')]
    app.run(main, argv=filtered_argv)

FATAL Flags parsing error: Unknown command line flag 'f'
Pass --helpshort or --helpfull to see help on flags.


AttributeError: 'tuple' object has no attribute 'tb_frame'